# Import

In [1]:
import tensorflow as tf
from tensorflow import keras
from matplotlib import pyplot as plt
import numpy as np
import pathlib
import os
import datetime

AUTOTUNE = tf.data.experimental.AUTOTUNE

In [2]:
print("tensorflow version check : ", tf.__version__)
print("gpu check : ",tf.test.is_gpu_available())

tensorflow version check :  2.0.0
gpu check :  True


# Data processing Pipeline

## Data 불러오기
### 이미지 파일들을 그대로 불러오면 용량이 너무 크니 경로를 지정하여 다룸

In [3]:
train_dir = pathlib.Path('../../before_folder/image_train/Training')
private_test_dir = pathlib.Path('../../before_folder/image_train/PrivateTest')
public_test_dir = pathlib.Path('../../before_folder/image_train/PublicTest')

In [4]:
# 각 경로 별 데이터 갯수 확인
print(len(list(train_dir.glob('*/*.jpg'))))
print(len(list(public_test_dir.glob('*/*.jpg'))))
print(len(list(private_test_dir.glob('*/*.jpg'))))

28708
3589
3589


#### dataset 만들기

In [5]:
list_ds = tf.data.Dataset.list_files(str(train_dir/'*/*.jpg'))
val_list_ds = tf.data.Dataset.list_files(str(public_test_dir/'*/*.jpg'))

In [6]:
#확인하기
for f in list_ds.take(5) : 
    print(f.numpy())

b'..\\..\\before_folder\\image_train\\Training\\Fear\\11916.jpg'
b'..\\..\\before_folder\\image_train\\Training\\Neutral\\12468.jpg'
b'..\\..\\before_folder\\image_train\\Training\\Sad\\3265.jpg'
b'..\\..\\before_folder\\image_train\\Training\\Neutral\\21528.jpg'
b'..\\..\\before_folder\\image_train\\Training\\Fear\\2177.jpg'


## Data labeling을 위한 전처리

In [7]:
CLASS_NAMES = np.array([item.name for item in train_dir.glob('*')])

CLASS_NAMES

array(['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise'],
      dtype='<U8')

In [8]:
CLASS_NAME_TO_ID = {}
for id, name in enumerate(CLASS_NAMES):
    CLASS_NAME_TO_ID[name] = id
    
print(CLASS_NAME_TO_ID)

{'Angry': 0, 'Disgust': 1, 'Fear': 2, 'Happy': 3, 'Neutral': 4, 'Sad': 5, 'Surprise': 6}


In [9]:
keys = list(CLASS_NAME_TO_ID.keys())
ids = list(CLASS_NAME_TO_ID.values())
print(keys,ids)

['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise'] [0, 1, 2, 3, 4, 5, 6]


In [10]:
table = tf.lookup.StaticHashTable(
    initializer= tf.lookup.KeyValueTensorInitializer(
    keys= tf.constant(keys),
    values=tf.constant(ids),),
    default_value=tf.constant(-1),
    name="class_weight")

In [11]:
BATCH_SIZE = 64
IMG_HEIGHT = 96
IMG_WIDTH = 96

## data labeling을 위한 함수 선언

In [12]:
def get_label(file_path) :
    parts = tf.strings.split(file_path, '\\')
    return table.lookup(parts[-2])

In [13]:
def decode_img(img):
    img = tf.image.decode_jpeg(img,channels=1)
    img = tf.image.convert_image_dtype(img, tf.float32)
    return tf.image.resize(img,[IMG_WIDTH,IMG_HEIGHT])

In [14]:
def process_path(file_path):
    label = get_label(file_path)
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    return img, label

### labeling 된 데이터셋 만들기

In [15]:
labeled_ds = list_ds.shuffle(10000).map(process_path, num_parallel_calls=AUTOTUNE).batch(BATCH_SIZE)
val_labeled_ds = val_list_ds.map(process_path,num_parallel_calls=AUTOTUNE).batch(BATCH_SIZE)

In [16]:
for image, label in labeled_ds.take(1) :
    print("image shape : ", image.numpy().shape)
    print("Label : ", label.numpy())
    
for i in val_labeled_ds.take(1):
    print('val_img_shape : ',i[0].shape)

image shape :  (64, 96, 96, 1)
Label :  [6 4 4 5 0 4 6 0 3 5 5 0 5 2 2 2 3 3 4 5 3 5 0 0 3 2 6 0 2 2 4 3 0 2 2 0 2
 5 3 4 6 2 0 3 4 0 5 2 2 6 4 3 4 0 5 4 6 5 5 5 6 5 2 4]
val_img_shape :  (64, 96, 96, 1)


# Model 만들기  

In [27]:
def create_model() :
    inputs = keras.Input(shape=(96, 96,1), name='img')

    feature = tf.keras.layers.Conv2D(96, 3, padding = 'same', input_shape=(IMG_HEIGHT, IMG_WIDTH ,3))(inputs)
    feature = tf.keras.layers.BatchNormalization()(feature)
    feature = tf.keras.layers.ReLU()(feature)
    feature = tf.keras.layers.Conv2D(16, 3, padding='same')(feature)
    feature = tf.keras.layers.BatchNormalization()(feature)
    feature = tf.keras.layers.ReLU()(feature)
    feature = tf.keras.layers.MaxPooling2D()(feature)
    feature = tf.keras.layers.Dropout(0.2)(feature)

    feature = tf.keras.layers.Conv2D(32, 3, padding='same')(feature)
    feature = tf.keras.layers.BatchNormalization()(feature)
    feature = tf.keras.layers.ReLU()(feature)
    feature = tf.keras.layers.MaxPooling2D()(feature)
    feature = tf.keras.layers.Dropout(0.2)(feature)

    feature = tf.keras.layers.Conv2D(64, 3, padding='same')(feature)
    feature = tf.keras.layers.BatchNormalization()(feature)
    feature = tf.keras.layers.ReLU()(feature)
    feature = tf.keras.layers.MaxPooling2D()(feature)
    feature = tf.keras.layers.Dropout(0.2)(feature)
    
    feature = tf.keras.layers.Conv2D(16, 3, padding='same')(feature)
    feature = tf.keras.layers.BatchNormalization()(feature)
    feature = tf.keras.layers.ReLU()(feature)
    feature = tf.keras.layers.MaxPooling2D()(feature)
    feature = tf.keras.layers.Dropout(0.2)(feature)
    
    #feature = tf.keras.layers.Flatten()(feature)
    feature = tf.keras.layers.GlobalAveragePooling2D()(feature)
    feature = tf.keras.layers.Dense(96, activation='relu')(feature)
    feature = tf.keras.layers.Dense(32, activation='relu')(feature)
    outputs = tf.keras.layers.Dense(7, activation='softmax')(feature)

    model = keras.Model(inputs=inputs, outputs=outputs)
    
    return model

In [28]:
model = create_model()

In [29]:
train_len = len(list(list_ds))
val_len = len(list(val_list_ds))

steps_per_epoch = train_len // BATCH_SIZE
validation_steps = val_len // BATCH_SIZE

In [30]:
checkpoint_path = "../../before_folder/model_save/cp-{epoch:04d}-{val_loss:.2f}.hdf5"

In [31]:
EPOCH = 200
learning_rate = 0.0001

def train_model(model):
    
    model.compile(optimizer= tf.keras.optimizers.Adam(learning_rate = learning_rate),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                                   period=5)
    
    history = model.fit(labeled_ds.repeat(),
                        epochs=EPOCH,
                        steps_per_epoch = steps_per_epoch,
                        validation_data = val_labeled_ds,
                        validation_steps = validation_steps,
                        callbacks = [model_checkpoint_callback])
    
    return history

In [ ]:
history = train_model(model)

Train for 448 steps, validate for 56 steps
Epoch 1/200
448/448 [==============================] - 31s 68ms/step - loss: 1.1174 - accuracy: 0.5753 - val_loss: 1.2048 - val_accuracy: 0.5312.1168 - 
Epoch 2/200
448/448 [==============================] - 29s 65ms/step - loss: 1.1089 - accuracy: 0.5797 - val_loss: 1.1538 - val_accuracy: 0.5675
Epoch 3/200
448/448 [==============================] - 29s 65ms/step - loss: 1.1159 - accuracy: 0.5766 - val_loss: 1.1513 - val_accuracy: 0.5611
Epoch 4/200
448/448 [==============================] - 29s 65ms/step - loss: 1.1122 - accuracy: 0.5770 - val_loss: 1.3189 - val_accuracy: 0.5109
Epoch 5/200
448/448 [==============================] - 29s 66ms/step - loss: 1.1046 - accuracy: 0.5804 - val_loss: 1.1858 - val_accuracy: 0.5472
Epoch 6/200
448/448 [==============================] - 30s 68ms/step - loss: 1.1100 - accuracy: 0.5752 - val_loss: 1.2178 - val_accuracy: 0.5257
Epoch 7/200
448/448 [==============================] - 30s 67ms/step - loss: 1.

# Model Test

In [20]:
model.evaluate(val_labeled_ds)

57/57 [==============================] - 4s 74ms/step - loss: 1.7269 - sparse_categorical_accuracy: 0.4851


[1.726860144682098, 0.48509336]